In [2]:
!pip install langchain chromadb gradio 
!pip install -U langchain-community

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   -------

  You can safely remove it manually.


In [ ]:
#!pip install langchain chromadb gradio pymupdf
#!pip install -U langchain-community
import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import ollama
import re
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=100
    )
    chunks = text_splitter.split_documents(data)
    embeddings = OllamaEmbeddings(model=“deepseek-r1:1.5b”)
    vectorstore = Chroma.from_documents(
        documents=chunks, embedding=embeddings, persist_directory=“./chroma_db”
    )
    retriever = vectorstore.as_retriever()
    return text_splitter, vectorstore, retriever
def combine_docs(docs):
    return “\n\n”.join(doc.page_content for doc in docs)
def ollama_llm(question, context):
    formatted_prompt = f”Question: {question}\n\nContext: {context}”
    response = ollama.chat(
        model=“deepseek-r1:1.5b”,
        messages=[{“role”: “user”, “content”: formatted_prompt}],
    )
    response_content = response[“message”][“content”]
    # Remove content between <think> and </think> tags to remove thinking output
    final_answer = re.sub(r”<think>.*?</think>“, “”, response_content, flags=re.DOTALL).strip()
    return final_answer
def rag_chain(question, text_splitter, vectorstore, retriever):
    retrieved_docs = retriever.invoke(question)
    formatted_content = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_content)
def ask_question(pdf_bytes, question):
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes)
    if text_splitter is None:
        return “No PDF uploaded.”  # Return a string message
    result = rag_chain(question, text_splitter, vectorstore, retriever)
    return result  # Return the string directly (no dictionary)
interface = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.File(label=“Upload PDF (optional)“),
        gr.Textbox(label=“Ask a question”),
    ],
    outputs=“text”,
    title=“Ask questions about your PDF”,
    description=“Use DeepSeek-R1 to answer your questions about the uploaded PDF document.“,
)
interface.launch(share=True, show_error=True)

In [5]:
import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import ollama
import re

def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=100
    )
    chunks = text_splitter.split_documents(data)
    embeddings = OllamaEmbeddings(model="deepseek-r1")
    vectorstore = Chroma.from_documents(
        documents=chunks, embedding=embeddings, persist_directory="./chroma_db"
    )
    retriever = vectorstore.as_retriever()
    return text_splitter, vectorstore, retriever

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(
        model="deepseek-r1:1.5b",
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    response_content = response["message"]["content"]
    # Remove content between <think> and </think> tags to remove thinking output
    final_answer = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()
    return final_answer

def rag_chain(question, text_splitter, vectorstore, retriever):
    retrieved_docs = retriever.invoke(question)
    formatted_content = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_content)

def ask_question(pdf_bytes, question):
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes)
    if text_splitter is None:
        return None  # No PDF uploaded
    result = rag_chain(question, text_splitter, vectorstore, retriever)
    return result

interface = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.File(label="Upload PDF (optional)"),
        gr.Textbox(label="Ask a question"),
    ],
    outputs="text",
    title="Ask questions about your PDF",
    description="Use DeepSeek-R1 to answer your questions about the uploaded PDF document.",
)
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e1b1252e27c8073152.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "c:\Users\vikra\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py", line 589, in _lazy_parse
    import pymupdf
ModuleNotFoundError: No module named 'pymupdf'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\vikra\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\vikra\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\vikra\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line